#Prepared by Tamal Acharya

In [ ]:
!pip install langchain-google-genai beautifulsoup4 pymupdf -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 39.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [ ]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.3 MB/s eta 0:00:00


In [ ]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.0/19.0 MB 88.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 67.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 83.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 9.9 MB/s eta 0:

In [ ]:
import requests
from bs4 import BeautifulSoup
import io
from google.colab import userdata
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings

from IPython import get_ipython
from IPython.display import display
# %%
#!pip install langchain-google-genai beautifulsoup4 pymupdf -q

# %%
import requests
from bs4 import BeautifulSoup
import io
from google.colab import userdata
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA

def get_text_from_url(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for bad status codes
        soup = BeautifulSoup(response.content, 'html.parser')
        text = soup.get_text(separator='\n', strip=True)
        return text
    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL: {e}")
        return None

def get_text_from_pdf_url(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        pdf_content = response.content
        pdf_file = io.BytesIO(pdf_content)
        import fitz  # PyMuPDF
        doc = fitz.open(stream=pdf_file, filetype="pdf")
        text = ""
        for page in doc:
            text += page.get_text()
        return text
    except requests.exceptions.RequestException as e:
        print(f"Error fetching PDF URL: {e}")
        return None
    except ImportError:
        print("Please install pymupdf: !pip install pymupdf")
        return None
    except Exception as e:
        print(f"Error processing PDF: {e}")
        return None




In [ ]:
# Configure the Google Generative AI API
API_KEY = 'your_own_api_key'  #please use your own Gemini API key
genai.configure(api_key=API_KEY)

# Initialize the models
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-001", google_api_key=API_KEY)
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=API_KEY)



In [ ]:
# %%
# --- RAG Implementation with Chunking and Retrieval ---

def advanced_rag_chat(query, context_text, llm, embeddings):
    """
    RAG function with text chunking and retrieval.
    """
    if not context_text:
        print("No context available for processing.")
        return llm.invoke(query).content # Just answer based on general knowledge

    # 1. Chunk the text
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200
    )
    texts = text_splitter.split_text(context_text)

    # 2. Create embeddings and a vector store
    # Using an in-memory Chroma vector store for simplicity
    vectorstore = Chroma.from_texts(texts, embeddings)

    # 3. Create a retriever
    retriever = vectorstore.as_retriever()

    # 4. Create a RetrievalQA chain
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff", # "stuff" chain type puts all retrieved docs into the prompt
        retriever=retriever,
        return_source_documents=True # Optional: return the retrieved documents
    )

    # 5. Run the chain with the query
    response = qa_chain({"query": query})

    return response['result']


In [ ]:
# %%
# Example Usage:
# Replace with your desired URL (can be a web page or a direct link to a PDF)
# For a web page:
# url = "https://www.cnn.com/2023/11/20/india/india-cricket-world-cup-final-defeat-modi-intl-hnk/index.html"
# For a PDF:
url = "https://arxiv.org/pdf/2309.11519.pdf"

print(f"Fetching content from: {url}")

if url.lower().endswith('.pdf'):
    context_content = get_text_from_pdf_url(url)
else:
    context_content = get_text_from_url(url)


if context_content:
    print("Context fetched successfully. You can now ask questions about the content.")
    while True:
        user_query = input("Ask a question about the content (or type 'quit' to exit): ")
        if user_query.lower() == 'quit':
            break
        if user_query.strip():
            # Use the advanced RAG function
            answer = advanced_rag_chat(user_query, context_content, llm, embeddings)
            print("\nAnswer:")
            print(answer)
            print("-" * 20)
else:
    print("Failed to fetch content. Cannot answer questions based on this URL.")

Fetching content from: https://arxiv.org/pdf/2309.11519.pdf
Context fetched successfully. You can now ask questions about the content.
Ask a question about the content (or type 'quit' to exit): summarise the pdf


<ipython-input-3-9a478a21b2e3>:35: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain({"query": query})



Answer:
Here's a summary of the provided text:

The document investigates a design methodology for a coherent receiver. It explores optical confinement in waveguide detectors, focusing on how the width of the BPD (𝑊𝑃𝐷) affects optical confinement. The document uses a LUMERICAL mode solver to analyze the fundamental TE mode in detectors with varying widths. It also discusses calculations for heterodyne coherent receivers, including formulas for optical power on photodiodes (PDs). Additionally, the document mentions implementing Particle Swarm Optimization, detailing how velocity and position of particles are updated within the swarm. Several parameters used for simulation are listed in tables.
--------------------
Ask a question about the content (or type 'quit' to exit): give in bullet points the summary

Answer:
Here's a summary of the provided text in bullet points:

*   The research focuses on optimizing bandwidth for BPDs (bandwidth photodetectors) on a generic InP platform, aimin

In [ ]:
# def get_text_from_url(url):
#     try:
#         response = requests.get(url)
#         response.raise_for_status()  # Raise an exception for bad status codes
#         soup = BeautifulSoup(response.content, 'html.parser')
#         text = soup.get_text(separator='\n', strip=True)
#         return text
#     except requests.exceptions.RequestException as e:
#         print(f"Error fetching URL: {e}")
#         return None

# def get_text_from_pdf_url(url):
#     try:
#         response = requests.get(url)
#         response.raise_for_status()
#         pdf_content = response.content
#         pdf_file = io.BytesIO(pdf_content)
#         import fitz  # PyMuPDF
#         doc = fitz.open(stream=pdf_file, filetype="pdf")
#         text = ""
#         for page in doc:
#             text += page.get_text()
#         return text
#     except requests.exceptions.RequestException as e:
#         print(f"Error fetching PDF URL: {e}")
#         return None
#     except ImportError:
#         print("Please install pymupdf: !pip install pymupdf")
#         return None
#     except Exception as e:
#         print(f"Error processing PDF: {e}")
#         return None



# # %%
# # Configure the Google Generative AI API
# API_KEY = 'YOUR_API_KEY'  # Replace with your actual Gemini API key
# genai.configure(api_key=API_KEY)

# # Initialize the models
# llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-001", google_api_key=API_KEY)
# embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=API_KEY)